# Deep MNIST for Experts

TensorFlow는 대규모 수치 계산을 수행하는 강력한 라이브러리입니다. 특히 딥(Deep) 뉴럴 네트워크를 구현하고 학습하는 작업에 사용할 수 있습니다. 이 튜토리얼에서는 TensorFlow 모델의 기본 빌딩 블록을 배우면서 딥 컨볼루션 MNIST 분류기를 구성합니다.

이 튜토리얼은 신경망과 MNIST 데이터 세트에 익숙하다고 가정합니다. 이것에 대한 배경지식이 없다면 초보자를 위한 튜토리얼을 먼저 읽기 바랍니다. 시작하기 전에 반드시 TensorFlow를 설치하십시오.

## About this tutorial

이 튜토리얼의 첫 번째 부분에서는 Tensorflow 모델의 기본 구현 인 mnist_softmax.py 코드에서 어떤 일이 일어나는지 설명합니다. 두 번째 부분은 정확성을 향상시키는 몇 가지 방법을 보여줍니다.

이 튜토리얼의 각 코드를 복사하여 Python 환경에 붙여 넣거나, 그냥 코드를 읽고 넘어가도 됩니다.

이 튜토리얼을 통해 아래 항목에 대해 배울 수 있습니다.
* 이미지의 모든 픽셀을 보면서 MNIST 숫자를 인식하는 모델 인 softmax 회귀 함수를 만듭니다. 
* Tensorflow를 사용하여 수천 가지 데이터를 "조사"하여 숫자를 인식하도록 모델을 학습합니다 (첫 번째 Tensorflow 세션을 실행하여 수행) 
* 테스트 데이터로 모델의 정확성을 확인합니다. 
* 결과를 향상시키기 위해 다중 컨볼루션 신경 네트워크(CNN)를 구축, 학습 및 테스트합니다.

## Setup

모델을 만들기 전에 먼저 MNIST 데이터 세트를 불러온 다음 TensorFlow 세션에 들어 가겠습니다.

### Load MNIST Data

이 튜토리얼의 코드를 복사하여 붙여 넣는 경우, 다음 두 줄의 코드를 사용하여 자동으로 데이터를 다운로드하고 읽습니다.

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


여기서 mnist는 훈련, 검증 및 테스트 세트를 NumPy 배열로 저장하는 간단한 클래스입니다. 또한 아래에 사용할 데이터에 대한 mini batches를 반복하는 함수를 제공합니다.

### Start TensorFlow InteractiveSession

TensorFlow는 매우 효율적인 C++ 백엔드를 사용하여 계산을 수행합니다. 이 백엔드에 대한 연결을 세션이라고합니다. TensorFlow 프로그램의 일반적인 사용법은 먼저 그래프를 만든 다음 세션에 올려 시작하는 것입니다

여기서 우리는 편리한 **InteractiveSession** 클래스를 대신 사용하겠습니다. 이것은 TensorFlow에서 코드 구조화를 보다 유연하게 만듭니다. 계산 그래프를 먼저 설계한 다음 실행하는 인터리브 연산을 제공합니다. 이는 특히 IPython과 같은 대화식 환경에서 작업 할 때 편리합니다. **InteractiveSession**을 사용하지 않는 경우 세션을 시작하고 그래프를 런치하기 전에 전체 계산 그래프를 작성해야합니다.

In [2]:
import tensorflow as tf
sess = tf.InteractiveSession()

### Computation Graph

파이썬에서 효율적으로 행렬 곱셈과 같은 값 비싼 수치 연산을 하기위해, 일반적으로 다른 언어로 구현 된 매우 효율적인 코드로 구현된 NumPy와 같은 라이브러리를 사용합니다. 불행하게도, 모든 작업을 파이썬으로 다시 전환하면 많은 오버 헤드가 발생할 수 있습니다. GPU 또는 분산 방식으로 계산을 실행하려면 데이터를 전송하는 데 비용이 많이 드는 경우가 특히 많습니다.

TensorFlow는 파이썬 밖에서도 복잡한 작업을 수행하지만 이러한 오버 헤드를 피하기 위해 한 걸음 더 나아갑니다. TensorFlow는 Python과 독립적으로 값 비싼 단일 작업을 실행하는 대신, Python 외부에서 상호 작용하는 그래프의 연산을 수행합니다. (이와 같은 접근법은 Theano나 Torch 같은 몇몇 다른 머신러닝 라이브러리에서도 볼 수 있습니다.)

따라서 파이썬 코드의 역할은 이 계산 그래프를 작성하고 그래프의 어느 부분을 실행해야 하는지 지시하는 것입니다. 자세한 내용은 "Getting Started"의 계산 그래프 섹션을 참조하십시오.

## Build a Softmax Regression Model

이 섹션에서는 단일 선형 레이어가있는 softmax 회귀 모델을 작성합니다. 다음 섹션에서 우리는 이것을 다중 계층 컨볼루션 네트워크가있는 softmax 회귀의 경우로 확장 할 것입니다.

### Placeholders

우선 입력 이미지와 출력 클래스에 대한 노드를 생성하여 계산 그래프를 작성합니다.

In [3]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

**x** 는 특정 변수가 아닙니다. 이것은 **placeholder** 이며 Tensorflow에 Run을 수행 할 때 특정 값이 입력됩니다. 

우리는 무수히 많은 MNIST 이미지를 입력으로 사용하기를 원합니다. 각각의 이미지는 784 차원의 벡터로 전개됩니다. 이것을 [None, 784] 형태의 2 차원 텐서 froting-point로 나타냅니다. 여기서 첫번째 축인 None은 차원의 크기가 임의 일 수 있음을 의미하며, batch size와 대응됩니다. 타겟 출력 클래스 또한 2 차원 텐서로 구성되며, 각 행은 해당 MNIST 이미지가 속하는 디지트 클래스(0부터 9까지)를 나타내는 10차원 noe-hot 벡터입니다.

placeholder에 대한 shape 인수는 선택 사항이지만 TensorFlow가 일치하지 않는 텐서 shape에서 발생하는 버그를 자동으로 잡을  수있게합니다.

### Variables

우리 모델의 가중치와 바이어스도 필요합니다. 이러한 요소를 위와 같은 추가 입력으로 처리한다고 생각 할 수도 있지만 TensorFlow는 이를 처리하는 더 좋은 방법을 제공합니다. **Variable** 은 TensorFlow의 상호 작용 그래프 내 수정 가능한 텐서입니다. 그것은 계산에 사용되거나 심지어 중간에 수정 될 수도 있습니다. 머신러닝 어플리케이션에서 일반적으로 모델 파라미터는 변수입니다.

In [4]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

**tf.Variable** 호출을 통해 각 매개 변수의 초기 값을 할당합니다. 이 경우 W와 b를 0으로 채워진 텐서로 초기화합니다. W는 784x10 행렬 (784 개의 입력 피처와 10 개의 출력이 있기 때문에)이고 b는 10 개의 클래스가 있으므로 10 차원 벡터입니다.

변수를 세션 내에서 사용하기 전에 아래와 같이 초기화해야합니다. 이 단계에서는 이미 지정된 초기 값 (이 경우에는 0으로 가득 찬 텐서)을 취하여 각 변수에 할당합니다. 이 작업은 모든 변수에 대해 동시에 수행 할 수 있습니다.

In [5]:
sess.run(tf.global_variables_initializer())

### Predicted Class and Loss Function

이제 회귀 모델을 구현 할 수 있습니다. 단 한 줄이면 됩니다. 백터화 된 입력 이미지 x와 가중치 행렬 W를 곱한 다음, 바이어스 b를 더해줍니다.  

In [6]:
y = tf.matmul(x,W) + b

특정 손실 함수를 쉽게 지정할 수 있습니다. 손실은 단일 예에서 모델이 얼마나 잘못 예측 했는지를 나타냅니다. 우리는 모든 학습 데이터를 통해 학습하면서 이를 최소화하려고 노력합니다. 여기서 우리의 손실 함수는 목표와 모델의 예측에 적용되는 softmax 활성화 함수 사이의 교차 엔트로피입니다. 초보자 튜토리얼에서 봤듯이 안정된 공식을 사용합니다.

In [7]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

**tf.nn.softmax_cross_entropy_with_logits**는 정규화 되지 않은 예측값에 softmax를 내부적으로 취하고, 모든 클래스의 합을 계산합니다. 그리고  **tf.reduce_mean**은 이러한 합계에 대한 평균 값을 취합니다.

## Train the Model
이제 모델과 손실 함수를 정의 했으므로 TensorFlow를 사용하여 학습 할 차례입니다. TensorFlow는 전체 연산 그래프를 알고 있으므로, 각 변수에 대한 그라디언트를 찾을 수 있습니다. TensorFlow에는 다양한 [최적화 알고리즘](https://www.tensorflow.org/api_guides/python/train#optimizers)이 있습니다. 여기서는 steepest gradient descent값을 0.5로 설정하여 교차 엔트로피 값을 줄이는 학습을 수행할 것입니다.

In [8]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

TensorFlow에서 위의 한 줄로 최적화에 관한 새로운 연산을 그래프에 추가할 수 있습니다. 이러한 작업에는 그라디언트를 계산, 매개 변수 업데이트, 업데이트 단계를 매개 변수에 적용하는 작업이 포함되었습니다.

반환 된 작업 **train_step**을 실행(Run)하면 그라디언트 디센트 업데이트가 매개 변수에 적용됩니다. 따라서 train_step을 반복적으로 실행하여 모델 학습을 수행 할 수 있습니다.

In [9]:
for _ in range(1000):
  batch = mnist.train.next_batch(100)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

각 학습 반복문 내에서 100 개의 학습 데이터를 로드합니다. 그런 다음 **feed_dict**를 사용하여 **train_step** 연산을 실행하여 placeholder 텐서 x 및 y_에 학습데이터를 할당합니다. feed_dict를 사용하여 계산 그래프에서 임의의 텐서에 값을 할당 할 수 있습니다. 단지 placeholder에만 국한되지 않습니다.

### Evaluate the Model

우리 모델은 얼마나 잘 동작할까요?

음, 우선 우리가 올바른 라벨을 예측했는지 알아 보겠습니다. **tf.argmax**는 어떤 축을 따라 텐서에서 가장 높은 엔트리의 인덱스를 제공하는 매우 유용한 함수입니다. 예를 들어, **tf.argmax (y, 1)**는 우리 모델이 각 입력에 대해 가장 가능성이 있다고 생각하는 레이블이고, **tf.argmax(y_, 1)**는 정답 레이블입니다. **tf.equal**을 사용하여 예측이 정답과 일치하는지 확인할 수 있습니다.

In [10]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

그 결과로 우리에게 이진 리스트를 줍니다. 어떤 부분이 올바른지 결정하기 위해 floting-point로 캐스팅 한 다음 평균을 취합니다. 예를 들어 $[True, False, True, True]$는 $[1,0,1,1]$이 되어, 평균 $0.75$가됩니다.
끝으로 테스트 데이터에 대한 정확도를 측정하겠습니다. 약 92%가 나올 것입니다.

In [12]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9194


## Build a Multilayer Convolutional Network

Getting 92% accuracy on MNIST is bad. It's almost embarrassingly bad. In this section, we'll fix that, jumping from a very simple model to something moderately sophisticated: a small convolutional neural network. This will get us to around 99.2% accuracy -- not state of the art, but respectable.

MNIST에서 92 % 정확도를 얻는 것은 좋지 않습니다. 그것은 아마 당황스러울 정도로 안 좋은 결과입니다. 이 섹션에서는 아주 단순한 기존 모델을 다소 정교한 것, 즉 간단한 콘볼루션 신경 네트워크로 고쳐 보겠습니다. 이것은 99.2 %의 정확도를 보일 것입니다. 최신 기술 수준은 아니지만 꽤 괜찮은 결과를 보여줍니다.

### Weight Initialization

이 모델을 만들려면 많은 가중치와 바이어스를 만들어야합니다. 일반적으로 대칭성을 무너뜨리고, 0 그래디언트 문제를 방지하기 위해 적은 양의 노이즈로 가중치를 초기화해야합니다. 우리가 [ReLU](https://en.wikipedia.org/wiki/Rectifier_(nural_networks) 뉴런을 사용하고 있기 때문에, "죽은 뉴런(dead neurons)" 문제를 피하기 위해 작은 양수값으로 바이어스로 초기화하는 것도 좋습니다. 모델을 구축하는 동안 반복적으로 수행되는 작업을 위한 두 가지 편리한 함수를 만들어 보겠습니다.

In [13]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

### Convolution and Pooling

TensorFlow는 또한 컨볼 루션 및 풀링 작업에서 많은 유연성을 제공합니다. 가령 바운더리를 어떻게 처리할지, 우리의 stride는 얼마로 할지를 정할 수 있습니다. 여기서는 항상 디폴트 버전을 선택합니다. 우리의 컨볼루션은 stride 값으로 1을 사용하며, 출력이 입력과 동일한 크기가 되도록 0으로 채워(zero padding)집니다. 풀링은 2x2 블록의 일반적인 max 풀링입니다. 코드를 보다 깔끔하게 유지하려면 이러한 연산을 함수로 추상화하는 것이 좋습니다. 

In [14]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

### First Convolutional Layer

이제 첫 번째 레이어를 구현할 수 있습니다. 컨볼 루션(convolution)과 최대 풀링(max pooling)으로 구성됩니다. 컨볼 루션은 각 **5x5** 패치에 대해 **32**개 피쳐를 연산합니다. 그 것의 가중치 텐서는 [5, 5, 1, 32]의 형태를 가질 것입니다. 처음 두 차원은 패치 크기이고, 다음은 입력 채널 수이며, 마지막은 출력 채널 수입니다. 각 출력 채널에 대한 component가 있는 바이어스 벡터도 있을 것입니다.

In [15]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

레이어를 적용하기 위해 먼저 x를 4d 텐서의 형태로 바꿉니다.  두 번째 및 세 번째 차원은 이미지 너비와 높이에 해당하고 최종 차원은 색상 채널 수에 해당합니다.

In [16]:
x_image = tf.reshape(x, [-1,28,28,1])

그런 다음 x_image를 체중 텐서와 곱하고, 바이어스를 추가한 다음, ReLU 함수를 적용하고, 마지막으로 최대 풀링을 적용합니다. max_pool_2x2 메소드는 이미지 크기를 **14x14**로 줄입니다.

In [17]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

### Second Convolutional Layer

딥 네트워크를 구축하기 위해, 위와 같은 유형의 레이어를 여러 개 쌓습니다. 두 번째 레이어에는 각 5x5 패치에 대해 64 개의 피처가 있습니다.

In [18]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

### Densely Connected Layer

이미지 크기가 **7x7 **로 축소되었으므로, 전체 이미지를 처리 할 수 있도록 **1024** 뉴런의 fully-connected 레이어를 추가합니다. 풀링 레이어에서 텐서를 벡터의 배치로 변형하고, 가중치 행렬을 곱하고, 바이어스를 더한 다음, ReLU를 적용합니다.

In [19]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#### Dropout

오버피팅을 줄이기 위해 [드롭 아웃](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf)을 readout 레이어 앞에 적용합니다. 우리는 드롭 아웃 중에 뉴런의 출력이 유지 될 확률에 대한 placeholder(keep_prob)를 만듭니다. 이를 통해 우리는 훈련 중 드롭 아웃을 켜고 테스트 중에 끌 수 있습니다. TensorFlow의 **tf.nn.dropout** op는 마스킹 외에도 스케일링 뉴런 출력을 자동으로 처리하므로 드롭 아웃은 추가 스케일링 작업 없이도 작동합니다.$^1$ 

In [20]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### Readout Layer

마지막으로 위의 softmax 회귀와 마찬가지로 마지막 레이어를 추가합니다.

In [21]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

### Train and Evaluate the Model

이 모델은 얼마나 잘 동작 할까요? 그것을 학습하고 평가하기 위해 위의 간단한 SoftMax 네트워크와 거의 동일한 코드를 사용할 것입니다.

차이점은 다음과 같습니다. 
* 우리는 그라디언트 디센트 최적화 출을 보다 정교한 **ADAM 최적화 툴**로 대체 할 것입니다. 
* 드롭 아웃 비율을 제어하기 위해 **feed_dict**에 추가 매개 변수 **keep_prob**를 추가합니다. 
* 교육 과정에서 100 회 반복마다 로깅을 추가합니다. 
언제든지 이 코드를 실행하고 자유롭게 진행할 수 있지만, 2만 건의 반복 학습을 수행하며 프로세서에 따라 다소 시간이 걸릴 수 있습니다(최대 30 분 소요).

In [22]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.06
step 100, training accuracy 0.84
step 200, training accuracy 0.92
step 300, training accuracy 0.92
step 400, training accuracy 0.94
step 500, training accuracy 0.98
step 600, training accuracy 0.92
step 700, training accuracy 0.94
step 800, training accuracy 0.96
step 900, training accuracy 0.94
step 1000, training accuracy 0.98
step 1100, training accuracy 0.94
step 1200, training accuracy 0.98
step 1300, training accuracy 1
step 1400, training accuracy 0.92
step 1500, training accuracy 0.96
step 1600, training accuracy 0.98
step 1700, training accuracy 0.96
step 1800, training accuracy 0.98
step 1900, training accuracy 0.96
step 2000, training accuracy 0.98
step 2100, training accuracy 0.94
step 2200, training accuracy 0.94
step 2300, training accuracy 1
step 2400, training accuracy 0.96
step 2500, training accuracy 1
step 2600, training accuracy 1
step 2700, training accuracy 1
step 2800, training accuracy 1
step 2900, training accuracy 0.98
step 3000,

ResourceExhaustedError: OOM when allocating tensor with shape[10000,32,28,28]
	 [[Node: Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](Reshape_3, Variable_2/read)]]
	 [[Node: Mean_4/_29 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_80_Mean_4", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Conv2D', defined at:
  File "C:\Program Files\Anaconda3\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Program Files\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Program Files\Anaconda3\lib\site-packages\traitle

이 코드를 실행 한 후 최종 테스트 셋 정확도는 약 99.2%가 나올 것 입니다. .

우리는 TensorFlow를 사용하여 상당히 정교한 심층 학습 모델을 빠르고 쉽게 작성, 학습 및 평가하는 방법을 배웠습니다.
1: 이 간단한 컨벌루션 네트워크의 경우 드롭 아웃이있는 경우와 없는 경우에 실제 성능은 거의 동일합니다. 드롭 아웃은 오버피팅을 줄이는 데 효과적이지만, 매우 큰 뉴럴 네트워크을 학습 할 때 가장 유용합니다.